In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
#Start Spark with BigDL support
from pyspark import SparkContext
import bigdl
import bigdl.util.common
sc = SparkContext.getOrCreate(conf=bigdl.util.common.create_spark_conf().setMaster("local[3]")
                              .set("spark.driver.memory","1g"))
bigdl.util.common.init_engine()

<!-- requirement: images/matrix.svg -->
<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/tensorboardcmd.py -->

# Intro to BigDL

BigDL is a relatively new package for Spark that allows deep learning, *i.e.* the training of large neural networks.  This is not possible in the normal Spark machine learning libraries, at least in practice&mdash;as networks grow, training becomes much slower and often does not converge correctly.  This is a problem with how derivatives are computed, among other things.  Recently, the way around this has been to perform as much ETL as possible in Spark, then load the data into a deep learning framework such as TensorFlow or Torch.  

With BigDL, this can now all be done in the same system, utilizing Spark's powerful data processing capabilities, then feeding the results directly in to a deep learning model in the same cluster, utilizing the same setup and Spark's distributed computing capabilities.

Similar to TensorFlow, Torch, etc., BigDL has multiple interfaces for setting up networks, allowing both very fine grained control and a simpler layer interface covering most common layouts.

### Math Kernel Library (MKL) and CPUs

Unlike many similar libraries, BigDL is optimized to run on CPUs.  Taking advantage of Intel's Math Kernel Library (MKL) to perform efficient vectorized operations, it can achieve GPU-like performance on commodity Intel CPUs.  MKL is an extremely optimized library for doing certain computations, particularly linear algebra&mdash;which is what Machine Learning algorithms and neural networks are based on.  You can read more about it on [Wikipedia](https://en.wikipedia.org/wiki/Math_Kernel_Library) and [Intel's website](https://software.intel.com/en-us/mkl)

## Machine Learning terminology

We're going to be working with neural networks to do deep learning, so we'll be using the language of Machine Learning (ML).  This is a quick refresher to make sure we are all familiar with the notation and terminology.

Since we'll be working with neural networks, we'll be doing _Supervised Learning_. (There is also _Unsupervised Learning_, which we won't discuss.)  In Supervised Learning, our goal is to build a predictive model that approximates the relationship between some independent variables, called features, and a dependent variable, called the label. A classic example is that you have data measuring information about houses, say ZIP code, median household income, crime rate, distance to public transport, pollution level, and the price that house sold for.  Using that, you want to predict what price you can sell your house for.

An _observation_ is a single data point where we have a value for each variable.  We'll write $x$ to denote values of the features associated with an observation and $y$ to denote the value of the associated label.  A labeled data set is just a collection of pairs $(x,y)$.  Given a training data set, our goal is to find a function, also called a model,

$$y_{pred} = f(x)$$

that gives $y_{pred}$ as close as possible to the $y$'s we already know about.  Then, when we get an $x$ where we _don't_ know the $y$, say the house that we are thinking about selling, we can hopefully get a good prediction of what to expect.  We'll talk about how to find this function later, but first let's be more specific about what $x$ and $y$ are.

In ML problems, we will be looking at data that consists of vectors of numbers, for example

```x = [0.2, 0.4, -1.4, 7.9, ...]
```

These vectors can come from nearly any source, and we may need to do some transformations to make it look like this, but this is mandatory - we're going to be working with models that use lot of linear algebra and they require vectors as input.  We'll write $\vec{x}$ to emphasize that $x$ is a vector and $x_j$ to denote its $j$th component (the $j$th feature).  When working with several observations, we'll write $\vec{x}_i$ to denote the features of the $i$th observation and $x_{ji}$ to denote the $j$th feature of the $i$th observation.  

We usually stack the feature vectors $\vec{x}_i$ and label values $y_i$ to form a feature matrix $\mathbf{X}$ and a label vector $\vec{y}$.  The feature matrix $\mathbf{X}$ has dimensions $n \times m$ where $n$ is the number of observations and $m$ is the number of features.  $\vec{y}$ is typically a $n \times 1$ column vector, but it will have more columns when the labels are vectors instead of scalars.  Together, $\mathbf{X}$ and $\vec{y}$ form our _training data set_.

![Feature matrix](images/matrix.svg)


Let's come back to that function from before now.  In a non-ML world, we would have an expert sit down, study the housing market, design an economic model, and hand craft an appropriate function $f(\vec{x})$.  This, of course, has a lot of problems, not the least of which is that we generally don't have a housing expert on-hand.  Instead, we're going to let the computer figure it out for us.  

We can leverage our training data by choosing a flexible family of functions $f(\beta, \vec{x})$.  This restricts the space from all possible functions to just some set, for example all the linear combinations of the features, or a binary tree based on them.  By varying the parameters $\beta$ (usually many of these) to give us the closest match to our desired function, we can search over this set of functions and choose the best one - in our examples, the best linear coefficients or the best splits to make at each node.  This reduces the problem to one of *minimization*: if we have a metric that quantifies how far each of our predictions is from the true label value, then we can define an associated "cost function" or "loss function". We then just look for the parameter values that give us the smallest cost.

$$C(\beta) = \sum_i \mathrm{metric}(f(\beta, \vec{x}_i), y_i)$$

We take the fitted $\bar{\beta}$ that minimizes this error, and can use that to make predictions going forward.  If we're given a new data point where we don't know the true label value, we can approximate it with

$$y_{pred} = f(\bar{\beta}, x_{new})$$

Everything else in (supervised) machine learning is details - what family of functions do we use?  What metric?  How do we do the minimization?  What optimizations do we need?  How do we deal with gigantic data sets?

### Metrics

Let's look at a few of those details.  First, we need to choose a metric to measure how well our model is doing at predicting our target values.  

#### Regression

If your target is a simple value, like the price of a house, you are doing _Regression_. Here, the metric is relatively clear&mdash;we need to measure the difference between our prediction and the target/label/true value (these terms are used interchangeably).  Since we'll be summing these up, we'll want to make sure we aren't getting negative differences.  This leads directly to the most common metric (also called error measurement), the mean square error (MSE):

$$C = \frac{1}{N} \sum_i (f(\beta, \vec{x}_i) - y_i)^2$$

That $1/N$ is just making it a mean instead of a straight sum.  Alternatively, you could take the absolute value to get the *L1* measurement or Absolute Error

$$C = \frac{1}{N} \sum_i \left| f(\beta, \vec{x}_i) - y_i \right|$$

but the MSE is much more nicely behaved mathematically and is usually what is used.

#### Classification

In _Classification_ our targets are discrete categories instead of continuous values.  Here our MSE metric doesn't make much sense - if you are categorizing pictures in to animal, vegetable, and mineral, what does (animal - mineral)$^2$ mean?

The first thought might be to dodge the issue and say that we'll just take animal as $1$, vegetable as $2$, and mineral as $3$, but then we have two different problems.  First, what happens when our model predicts $2.7$? Or worse, $4.2$?  Second, do we really want to say that saying animal when you meant mineral is four times as bad as saying you meant vegetable when you said mineral?

In these cases, we really want to do something different.  There are many solutions to this problem, but since we'll be working with neural networks, one in particular stands out: cross entropy.

In the case of just two classes (say, spam or not spam), we really can get away with predicting a single number: the probability that something is spam, denoted $p_i$.  If this number is high, we're pretty confident it's spam.  If it's low, we're pretty confident it's not.  Since it's a probability, we'll want it to be between $0$ and $1$.  With that restriction, and if we also have our labels be $0$ for "not spam" and $1$ for "spam", we can now define cross entropy as

$$C = -\sum_i y_i \log(p_i) + (1 - y_i) \log(1-p_i)$$

It looks a little complicated this way, but since $y_i$ is always $0$ or $1$, we only get one term for each observation: $\log(p_i)$ if it's a $1$, $\log(1-p_i)$ if it's a $0$.  We're just getting the log of how confident we are it's in the right category.

With many categories, we'll need to alter this slightly.  We're going to need to get a $p_{ik}$ for each observation for each category, but with a restriction - we don't just want _each_ of these to go from $0$ to $1$, we want each to be positive and their _sum_ to be $1$, so it's a proper "probability of being in class $k$."  

$$\sum_k p_{ik} = 1$$

Our cost function is now just an extension of the one before, taking $\log(p)$ for the correct category 

$$C = -\sum_i \log(p_{iy_i})$$

where $y_i$ is the true class for each observation.

**Questions:** 
1. What are some examples of classification problems? Regression problems?
1. When might the Absolute Error be a better choice than MSE?

### Linear Regression

We mentioned it before, but the simplest (non-trivial) regressor is the Linear Regressor.  It is just predicting the $y$ value to be the weighted sum of the features

$$y_{pred} = \vec{x} \cdot \vec{\beta} = \sum_j \beta_j x_j = \beta_0 x_0 + \beta_1 x_1 + \ldots$$

Where is the constant term, you might ask?  It's included in practice, but it ruins the symmetry of our equations, so we usually cheat in mathematical derivations by adding a "dummy feature" $x_0 = 1$ instead.  This makes our feature space one dimension larger than it was originally.

Alternatively, in matrix notation, we can write this as

$$\vec{y}_{pred} = \mathbf{X} \cdot \vec{\beta}$$

In 2D, this gives a very simple expression

$$y_{pred} = a x_1 + b x_2 + c$$

If we pick any constant value of prediction (say, asking for those pairs of feature values that give a prediction of zero, or 100), we just get a line.  Hence the "linear regression".

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

x = np.linspace(0, 10, 80)
y = x * (np.random.rand()+3) + np.random.randn(x.shape[0])*3

def plot_linear(slope=4.5):
    plt.figure()
    plt.plot(x, y, '.')
    plt.plot(x, slope * x)
    plt.title("MSE: %f" % np.mean((y - slope*x)**2))
    plt.axis((-0.5,10.5,-3,38))

interact(plot_linear, slope=(0, 7, .5));

### Logistic Regression

We can also modify this to get a classification algorithm that outputs probabilities. The linear regressor can generate any real number (just imagine holding all of the $x$'s except one constant, then varying that one over all possible values), so we can't interpret the outputs as probabilities directly.

But there's a nice function called the logistic (or sigmoid) function that maps any real number into the range $(0,1)$

$$\mathrm{sigmoid}(x) = \frac{\exp(x)}{1 + \exp(x)} = \frac{1}{1 + \exp(-x)}$$

In [ ]:
sigmoid = lambda x: np.exp(x) / (1 + np.exp(x))

lx = np.linspace(-10, 10)
ly = sigmoid(lx)
plt.plot(lx, ly)
plt.xlabel(r'$X$')
plt.ylabel(r'$P(X)$')

With this in hand, we can easily make our linear regressor in to a classifier, called _Logistic Regression_. (Yes, it says regression in the name but is actually a classifier&mdash;history leaves us with odd things sometimes.) For the two class case, where we only need a single probability, we're pretty much done.

$$p_i = \mathrm{sigmoid}\left(\vec{x}_i\cdot\vec{\beta}\right)$$

For the multiclass case, things are a touch more complicated.  There are two common methods of turning two-class classifiers in to multiclass ones: one-vs-one and one-vs-all.  In one-vs-one, we train a two-class classifier on each possible pair of classes, giving it the chose class as the $1$ case and the other class as the $0$ case.  With our three classes, we'd get: animal vs vegetable, animal vs mineral, vegetable vs mineral.  But with more classes, the number of pairs grows quickly (4 classes has 6, 5 has 10, etc).  This usually isn't done with neural nets, so we won't worry about it here.

In one-vs-all, we train one classifier for each class, with the chosen class being $1$ and all the other classes being $0$.  This gives us a $p$ for each class k, then we need to normalize so they sum to one

$$p_{ik} = \frac{\mathrm{sigmoid}\left(\vec{x}_i\cdot\vec{\beta}_k\right)}
{\sum_m \mathrm{sigmoid}\left(\vec{x}_i\cdot\vec{\beta}_m\right)}$$

Instead of going this route, usually people directly apply the generalized logistic function, called softmax

$$p_{ik} = \frac{\exp\left(\vec{x}_i\cdot\vec{\beta}_k\right)}{\sum_m \exp\left(\vec{x}_i\cdot\vec{\beta}_m\right)}$$

The two behave similarly enough that it doesn't make much difference, but softmax is more mathematically correct (and better computationally).

**Questions**
1. What are the strengths and weaknesses of using a linear regressor?
1. When will a logistic regressor work well?  When will it work poorly?

In [ ]:
n = 1000
X = np.random.uniform(size=(n, 2))
C = (sigmoid(X.dot([2, -1]) - 0.5) + 0.1 * np.random.normal(size=n) > 0.5)

def plot_logistic(slope=1, intercept=0):
    xx, yy = np.meshgrid(np.linspace(-0.1,1.1,100), np.linspace(-0.1,1.1,100))
    cc = (xx*slope - yy + intercept > 0)
    plt.imshow(cc, extent=(-0.1,1.1,-0.1,1.1), origin='lower', cmap=plt.cm.bwr, vmin=-1, vmax=2)
    plt.scatter(*X.T, c=C, cmap=plt.cm.bwr, s=10)
    P = sigmoid(X.dot([slope, -1]) + intercept)
    plt.title("Cross Entropy: %f" % -(np.mean(C * np.log(P)) + np.mean((1-C) * np.log(1-P))))
    
interact(plot_logistic, slope=(0.,5.), intercept=(-1.,1.));

### Minimization

So far we've been adjusting the parameters by hand.  This is clearly inefficient and defeats the purpose of "letting the computer figure it out."  We need a minimizer.  This is actually a fairly well developed area of mathematics and computer science, and there are many very good ones out there.  They mostly won't work for us, however, as they will have problems with the large neural networks we'll be building later.  Instead, we're going to use one of the simplest, Stochastic Gradient Descent (SGD).

At its heart is a bit of calculus. (Don't worry if your calculus is rusty or non-existent&mdash;you can safely skip this part and just know that SGD does, in fact, find minimums of functions.) The gradient (a generalization of the derivative to multiple dimensions) gives us the direction in which a function is _increasing fastest_.  We'll want to go in the opposite direction.  Since the derivative itself varies from place to place, we'll just take a small step in that direction, then compute again, and repeat process.  As an algorithm:

1. Start with some initial guess for $\bar{\beta}$, call it $\beta_0$
1. For some number of steps:
    1. Compute $\nabla C(\beta_i)$
    1. Update our guess to $\beta_{i+1} = \beta_i - \eta \nabla C(\beta_i)$
1. Return the final $\beta_n$ as our best guess for $\bar{\beta}$

You'll note two things left ambiguous there&mdash;we now have an $\eta$, and we loop "for some number of steps."  The $\eta$ is a measure of how small our steps will be.  Selecting the correct one for a problem can be tricky.  If it's too small, you'll need many iterations of the loop to find a good value.  If it's too big, you can overshoot the minimum and wind up moving away from it.  The correct number of steps will depend heavily on what you're doing.  Ideally, you'd go until your update of $\beta$ was smaller than some amount, so you'd know the process had converged.  In practice, this is not a good idea with a big neural network, as it will (a) take a very long time and (b) probably not give you a better result than stopping earlier than that.  Instead, we usually just run for some number of iterations and see how we're doing, then run some more if we think we need to (we'll come back to this later).

The algorithm just described is actually just Gradient Descent.  The "Stochastic" part comes in when we make a slight change.  What is the $C(\beta)$ in the function above?  Presumably it's the cost function, such as the MSE for a linear regression problem.

$$C(\beta) = \frac{1}{N}\sum_i (\vec{x}\cdot\vec{\beta} - y_i)^2 = 
\frac{1}{N}\sum_i (\beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + ... - y_i)^2 = 
\frac{1}{N}\sum_i \left(\sum_j \beta_j x_{ij} - y_i\right)^2$$

Taking the gradient, we get a vector with a component for each $\beta_k$

$$\frac{\partial C}{\partial \beta_k} = 
\frac{1}{N}\sum_i \left(\sum_j 2 \left(\vec{x}\cdot\vec{\beta} - y_i\right) x_{ik}\right)$$

Note that the outer sum over runs over all observations.

This is what changes in SGD: For each evaluation of the derivative, we use some smaller set, called the _batch_ or _minibatch_.  Each run of the loop will get a different minibatch, so that all of the data gets used eventually.  This has two effects: it makes the computation much faster, especially for more complicated models, and it adds noise to the process.  That might _sound_ bad, but it's actually good, as it forces the model to explore more possible $\beta$ values and not get stuck in a local minimum, a place where the gradient is small or zero, but that isn't the global minimum.  The exact size to use is a balance between taking advantage of parallelism, the number of batches to run, wanting some noise, but not wanting too much.  Something in the low 100's is often a good choice on a smaller system.

Two terms come up in minibatching: iterations and epochs.  An _iteration_ is one run of the loop above - one gradient computation, one parameter update.  An _epoch_ is enough iterations that we've seen all the data once, usually a number of iterations equal to the data size over the batch size

$$n_{epoch} = \mathrm{ceil}\left(\frac{n_{obs}}{n_{batch}}\right)$$

In BigDL, we'll generally control the "for a while" aspect of SGD with a number of iterations or epochs.

In [ ]:
from ipywidgets import FloatSlider

def plot_sgd(eta=0.00005):
    beta = 1

    betas = [beta]
    errors = [np.mean((x * beta - y)**2)]

    for _ in range(10):
        gradient = np.sum(2 * (x * beta - y) * x)
        beta = beta - eta * gradient
        betas.append(beta)
        errors.append(np.mean((x * beta - y)**2))

    plt.subplot(121)
    plt.plot(errors)
    plt.xlabel('step')
    plt.ylabel('error')
    plt.ylim(0,None)
    plt.subplot(122)
    plt.plot(betas, errors)
    plt.xlabel(r'$\beta$')
    plt.ylim(0,None)
    plt.tight_layout()
    
interact(plot_sgd, eta=FloatSlider(min=0.00005, max=0.0004, step=0.00005, readout_format='.5f'));

### Tensors

Another detail - what happens if our data makes more sense as a matrix?  This is often the case with something that is intrinsically 2D, like the pixels in an image (which is something we'll be looking at).  This doesn't seem like a big adjustment, really - we just replace our vector $\vec{x}_i$ with a matrix $\mathbf{X}_i$.  But what is our collection of observations now?  We had a stack of vectors before, now we have a stack of matrices.  This is a mathematical object called a _Tensor_.

For our purposes, a tensor can be thought of as a generalization of a matrix.  A vector is a one-dimensional array of numbers and a matrix is a two-dimensional array.  A tensor then is an *n*-dimensional array of numbers.  Both vectors and matrices are just special cases of tensors.  (Mathematicians may quibble with this definition, insisting that tensors are defined by how they transform under coordinate transformations.  They have a point, but one that doesn't matter for us.)

They work pretty much the same as a matrix, but now you can have three (or four, or five, ...) indices.  So instead of $X_{ij}$ specifying a location, you'll need $X_{ijk}$ or $X_{ijkl}$.  And just as removing an index of a matrix gives you a vector, where 

$$X_{i\cdot} = \vec{x}_i$$

removing an index gives you a tensor of one dimension lower&mdash;a 3D tensor gives a matrix, etc.

$$X_{i\cdot\cdot} = \mathbf{X}_i$$

These are already implemented in python as `ndarray`, as we'll see shortly.

## BigDL and data

To be able to do any machine learning, we're first going to need to read in some data.  As is usually the case with working with PySpark, we're going to have to think a bit about classes. Python is fairly loose about them, but the underlying Scala engine is most definitely not. As per usual, we'll use NumPy as a bridge between the two, but we'll need to use some BigDL specific classes as well.

BigDL models data as tensors, much as TensorFlow does&mdash;this turns out to be a pretty natural formatting for deep learning.  These already have an implementation in NumPy, but let's take a moment to discuss what they are.

In the Scala version, we need to use the BigDL specific `Tensor` class for this.  In NumPy, this is just the `ndarray` type.  It's an *n*-dimensional array of numbers of a single type (`float`, `int`, etc.). 

In [ ]:
import numpy as np
twoD = np.array([[1,0,0],[0,1,0]])
print(twoD)

In [ ]:
print(twoD.shape)

In [ ]:
threeD = np.array([[[1,0,0,0],[0,1,0,0],[0,0,1,0]], 
                   [[1.1, 2.1, 1.2, 1.2], [2.0, 2.1, 2.3, 2.4], [3.4, 3.1, 3.3, 3.0]]])
print(threeD)

Note that NumPy correctly figures out they should all be floats, even though the first groups are integers.

In [ ]:
print(threeD.shape)

We'll be focusing on supervised learning, so we'll need more than just feature data&mdash;we're also going to need labels.  BigDL requires us to bundle them together in the `Sample` class.  Let's make a sample `Sample` of fake data.  To do this, we'll use the `Sample.from_ndarray` function.

We'll do this to set up a simple classification problem: we have three populations of points on two features (so we can make plots) and want to build a model that can tell them apart.

In [ ]:
# Generate some random data
import matplotlib.pyplot as plt

#We probably should shuffle this data to avoid biasing our minibatches,
#but this set is so simple it doesn't matter
centers = np.array([[0, 0]] * 150 + [[1, 0.5]] * 150 + [[0,1]]*150)
np.random.seed(41)
data = np.random.normal(0, 0.2, (450, 2)) + centers
labels = np.array([[1]] * 150 + [[2]] * 150 + [[3]]*150)

plt.scatter(data[:,0], data[:,1], c=labels.reshape(-1), cmap=plt.cm.brg)
plt.colorbar();

### Sample class
We now have two separate tensors (a matrix and a vector, in this case), one that has rows of feature vectors and one that has rows that are integer labels. We'll need to convert this to an RDD of `Sample`, one for each data point.  We do this with the `Sample.from_ndarray` function.  It acts on the level of observations&mdash;we will need an `RDD[Sample]` as input.

Here our data is quite simple, being a 2D `ndarray` (a matrix or a 2-tensor).  So we'll just need to join each of our feature vectors with their partner label to make a `Sample`.  We could make the `Sample` first then put that result in to an `RDD`, but we'll instead make the `RDD` first since that's more like what real data will be.  Note that both the data and the label have to be in `ndarray`s, even though those for the labels will be just one element long.

In [ ]:
from bigdl.util.common import Sample

data_with_labels = zip(data, labels)
samples = sc.parallelize(data_with_labels).map(lambda x: Sample.from_ndarray(x[0],x[1]))

In this case, we're using parallelize to take an existing iterable and turn it in to an RDD.  Real data, of course, will be coming from elsewhere and will most likely already be an RDD at this point.

In [ ]:
samples.take(5)

## Building a model

Now that our data is in the right form, we're going to start simple and build a logistic regressor.  Since there are three classes, we'll need to utilize multiclass techniques, but this is actually very easy with neural networks and BigDL.  The default behavior is one-vs-all, so we'll take advantage of that.

We're going to need three linear regressors, each with two weights and a bias.  Our three equations for each data point equations will be

$$y_1 = W_{11} x_1 + W_{21} x_2 + b_1\\y_2 = W_{12} x_1 + W_{22} x_2 + b_2\\y_3 = W_{13} x_1 + W_{23} x_2 + b_3$$

Note that the labels were designed to be suggestive: we can also write this in matrix form as

$$\vec{y}_i = \vec{x}_i\cdot\mathbf{W} + \vec{b}$$

Thus, we'll have a $2\times3$ matrix of weights, and a $3$ vector of biases.  We could also absorb the bias in to the weights as before, but we'll keep them separate for now.

This behavior is accomplished in BigDL with the `layer.Linear` class.  It actually does more general neural networks, and is equivalent to the "dense" layer in TensorFlow, but for now we can take it just as a linear regressor that we tell it how many inputs and how many outputs, or, equivalently, the shape of the weight matrix.

After this, we need merely take the output and route it through a softmax to get our $p$'s for each observation and category.

In BigDL, there are two different ways to construct a model: Sequential and Functional.  We'll build this model in both ways so we can compare.

### Sequential

The `Sequential` method of building does something that is very traditionally imperative&mdash;we build a container and give it a list of operations to perform, in order.  Here we only have two: the `Linear` model and the `SoftMax`.

In BigDL, these operations are all `layer`s: they take the output of the previous layer and do something to it, and you can build up a stack of them (and more complicated geometries).  We'll mostly use this method, as it has better support.

In [ ]:
#Contains all of the layers we'll be using
from bigdl.nn import layer

#Note: you are mutating state here, this is happening *outside* of Spark so it's OK
#What we're building is an operation for Spark to perform later, as per normal
model = layer.Sequential()
model.add(layer.Linear(2,3))
model.add(layer.SoftMax())

That's it.  The weights are initially set to random values (for good reason), but we can force them to a particular value if we wish.  Let's take a peek at them, then set them to a particular value to make sure we're getting what we expect.  We can also control how they're initialized, which will be discussed later.

In [ ]:
print(model.get_weights())

In [ ]:
model.set_weights([np.array([[0.1,0.2],[-0.1,0.1],[0.2,0.3]]), np.array([0.2,0.2,0.2])])
print(model.get_weights())

The `forward` method runs an input through the model without changing it.  We can run a single input...

In [ ]:
print(model.forward(np.array([1,1])))

...or an array of inputs.

In [ ]:
print(model.forward(np.array([[1,1],[0.4,0.3]])))

### Functional

The alternative is to use the functional interface.  It is significantly more flexible than the linear interface, and allows for building more complex models, at the cost of a slightly more complicated interface.  Each layer will have its own variable, and we'll tell BigDL which ones are input and output..

In [ ]:
# empty second () indicates taking in data
lin_f = layer.Linear(2,3)()
softmax_f = layer.SoftMax()(lin_f)

# defines the model, first list is input layers, second is ouput
# can have more than one in each
model_f = layer.Model([lin_f],[softmax_f])

## Optimizer

Now that we have a model, we'll want to train it. We need to find the parameters (in this case, $W$ and $b$) that give us the best results.  To do this, we choose a loss function, and we minimize this loss with respect to $W$ and $b$.  We'll use Stochastic Gradient Descent (SGD) and its derivatives (`Adam`, `AdaGrad`, `RMSProp`) since they're the most common and are built in to BigDL.

Loss functions are implemented in `bigdl.nn.criterion`, and there are several to select from.  Cross entropy as we defined it above is implemented by `ClassNLLCriterion` - this is actually a fairly sensible name, as those who have studied Bayesian statistics may have recognized cross entropy as a negative log likelihood.  There is also a `CrossEntropyCriterion`, but this implements both softmax and cross entropy.  If we wanted to use this, we'd need to leave off the `SoftMax` layer.  There are also regression losses built in such as `MSECriterion`.  The full list is available at [BigDL Loss functions](https://github.com/intel-analytics/BigDL/blob/master/docs/docs/APIGuide/Losses.md)

One funny wrinkle of `ClassNLLCriterion` is that it needs to know if you're giving it $p$ or $\log(p)$, as there is both a `SoftMax` and a `LogSoftMax` available.  This is controlled by the parameter `logProbAsInput` - set it to true (the default) if you've used `LogSoftMax`, false if you used `Softmax`.  For computational reasons, `LogSoftMax` is a little better, hence why it's the default.  One additional thing to notice: we used classes numbered $1$, $2$, ... This is actually _mandatory_ for using the `ClassNLLCriterion` (and `CrossEntropyCriterion`).  Of course, it's easy enough to map our classes to these, but there are other criterion that have different restrictions.  

If you do forget and have a class numbered $0$, you will get an extremely cryptic `BoxedError`.  This is actually a generic error, indicating that a `Future` call failed&mdash;it's simply saying that a Spark job returned as an error.  If you see a `BoxedError`, this is likely your culprit.

We now need to select an algorithm to do the minimization.  We'll use SGD, since it's the simplest and easiest to understand. In the future, we'll use some of the others, as they tend to work better.

With those two selected, we'll make an `Optimizer`, the function that actually _runs_ the optimization.  It takes as arguments the model, our `RDD` to train on, what criterion we're using, what optimizer to run, when to stop (the `end_trigger`), and our minibatch size.  All available optimizers are in `bigdl.optim.optimizer`. SGD is helpfully named `SGD`, and we have to give it an $\eta$. (We can also give it other parameters to fine tune its behavior.)

The `end_trigger` options are also in `bigdl.optim.optimizer`.  There are many of these, which can trigger when you hit a certain number of epochs (`MaxEpoch`), a certain number of iterations (`MaxIteration`), a certain value of the loss function (`MinLoss`), and a few others.  We're going to use the first two throughout this course.

We are a bit restricted in BigDL as to what we can use as batch sizes, as they must be a multiple of the number of workers (the number in the bracket in our `setMaster("local[n]")` when we start Spark)

In [ ]:
from bigdl.nn import criterion 
from bigdl.optim import optimizer


# These arguments are actually in their correct positional order, so the
# model=, training_rdd= ... could be left off. We leave them in for clarity.
# Note that logProbAsInput=False tells it we used Softmax rather than LogSoftmax
# Note that SGD requires that eta argument.
fitter = optimizer.Optimizer(model=model, training_rdd=samples, 
                             criterion=criterion.ClassNLLCriterion(logProbAsInput=False), 
                             optim_method=optimizer.SGD(0.05), end_trigger=optimizer.MaxEpoch(20), 
                             batch_size=60)

# Add in some tracking for later. We'll use these to force a unique name
# for our tracking files.
import time
from datetime import datetime, timedelta

# Tracking
now_string = datetime.now().strftime("%Y%m%d-%H%M%S")
trainSummary = optimizer.TrainSummary("./logs", "simple_class_{}".format(now_string))
trainSummary.set_summary_trigger("Loss", optimizer.EveryEpoch())
fitter.set_train_summary(trainSummary)

# Add checkpointing
fitter.set_checkpoint(optimizer.SeveralIteration(50), './logs/simple_class_checkpoint', 
                      isOverWrite=False)

Now we just run it!

In [ ]:
%%time
# The time magic tells us how long the cell took to run
trained_model = fitter.optimize()

If we run it again, it immediately exits since it's already hit its end_trigger.

In [ ]:
%%time
fitter.optimize()

## Making predictions

Now for the moment of truth&mdash;we have a trained model, and we need to see how well it did.  We can get predictions out of it with the predict method, which either takes an `RDD` or an `ndarray`. The latter is good for spot-testing things or feeding new, small data sets.

We don't really want to look at every prediction individually, so we'll look at the accuracy, the fraction it got right.  This is calculated by this helper function

In [ ]:
def get_accuracy(predicts, trues):
    return sum([int(predicts[i] == trues[i]) for i in range(len(predicts))]) * 1.0 / len(trues)

In [ ]:
predictRDD = trained_model.predict(samples)
print(predictRDD.take(5))

We're getting a triplet for each input, the $p$ for each category.  To determine the right one, we just ask for the one that has the largest value - all the first one for these first few values.  We'll do that with `argmax` on the NumPy array.  We'll need to remember to add $1$ to each value, since our classes are $1$, $2$, $3$ and `argmax` will return $0$, $1$, $2$.

In [ ]:
predictions = predictRDD.map(lambda x: x.argmax() + 1).collect()
print(predictions[:5])

In [ ]:
get_accuracy(predictions, labels)

Not bad.  With only two features, we can visualize what's going on here pretty easily.  Since our model is just three linear functions run through a softmax, we can plot the three lines.  First, let's grab those now-trained weights.

In [ ]:
weights = trained_model.get_weights()
print(weights)

Each set of weights plus a bias determines a linear function.  We can visualize each function by plotting a line of constant value. (This would be a line of constant $p$ for a two-class case.)  Let's go ahead and choose to the value corresponding to $0$, so we get

$$0 = W_{1m} x_1 + W_{2m} x_2 + b_m$$

If we take $x_1$ as our horizontal axis ("x") and $x_2$ as our vertical ("y"), we see we just have a line

$$y = -\frac{W_{1m}}{W_{2m}} x - \frac{b_m}{W_{2m}}$$

We'll also color the background based on what our model predicts for a given region

In [ ]:
# Set up a 100x100 grid of points across our space, predict on those to
# get our background coloring
mesh = np.column_stack(a.reshape(-1) for a in np.meshgrid(np.r_[-0.5:1.6:100j], np.r_[-0.5:2:100j]))
predict_mesh = trained_model.predict(mesh).argmax(axis=1)+1

In [ ]:
weights1 = weights[0][0]
bias1 = weights[1][0]

weights2 = weights[0][1]
bias2 = weights[1][1]

weights3 = weights[0][2]
bias3 = weights[1][2]

plt.imshow(predict_mesh.reshape(100,100), cmap=plt.cm.brg, alpha=0.5, origin='lower',
           extent=(-0.5, 1.6, -0.5, 2), vmin=1, vmax=3)

plt.scatter(data[:,0], data[:,1], c=labels.reshape(-1), cmap=plt.cm.brg, edgecolors='black')
plt.colorbar();
xx = np.linspace(-0.5,1.6,100)
yy1 = -weights1[0] / weights1[1] * xx - bias1 / weights1[1]
yy2 = -weights2[0] / weights2[1] * xx - bias2 / weights2[1]
yy3 = -weights3[0] / weights3[1] * xx - bias3 / weights3[1]
plt.plot(xx,yy1,xx,yy3,xx,yy2)
plt.ylim(-0.5,2)
plt.show()

### Tracking

You may have noticed in the optimizer step we added a `TrainSummary` object.  This is completely optional, but it allows us to see how the training progressed.  It tracks the loss over iterations of SGD by default.  There is an option available to also tracks all the weights by replacing "Loss" with "Parameters" in the summary setup, but these values are currently not easy to access (as of BigDL 0.5.0).  The lines that added it are

```
trainSummary = optimizer.TrainSummary("./logs", "simple_class_{}".format(now_string))
trainSummary.set_summary_trigger("Loss", optimizer.EveryEpoch())
fitter.set_train_summary(trainSummary)
```

This creates the summary by specifying a log directory, and the particular subdirectory for this optimization run.  Then we set a `Trigger`&mdash;this is just like the trigger we used in the optimizer itself, but instead of firing once, we want one that will fire repeatedly.  Those are `SeveralIteration` and `EveryEpoch`. There currently isn't support for several epochs, but you can emulate it with a judicious choice of iterations.  Lastly, we just add that to our fitter object.

In [ ]:
# Order: step, value, timestamp
losses = np.array(trainSummary.read_scalar('Loss'))
plt.plot(losses[:,0],losses[:,1])

We are clearly not done training; the loss function is still decreasing quite a bit.  That jumpiness is the "stochastic" part of SGD.  Let's change the end condition on the optimizer and tell it to train longer.

In [ ]:
%%time
#Train a bit longer, see what happens
fitter.set_end_when(optimizer.MaxEpoch(200))
fitter.optimize()

losses = np.array(trainSummary.read_scalar('Loss'))
plt.plot(losses[:,0],losses[:,1])

Much better&mdash;that looks pretty converged.  Let's see how it does.

In [ ]:
predictRDD = trained_model.predict(samples)
predictions = predictRDD.map(lambda x: x.argmax() + 1).collect()
get_accuracy(predictions, labels)

In [ ]:
weights = trained_model.get_weights()

weights1 = weights[0][0]
bias1 = weights[1][0]

weights2 = weights[0][1]
bias2 = weights[1][1]

weights3 = weights[0][2]
bias3 = weights[1][2]

mesh = np.column_stack(a.reshape(-1) for a in np.meshgrid(np.r_[-0.5:1.6:100j], np.r_[-0.5:2:100j]))
predict_mesh = trained_model.predict(mesh).argmax(axis=1)+1
plt.imshow(predict_mesh.reshape(100,100), cmap=plt.cm.brg, alpha=0.5, origin='lower',
           extent=(-0.5, 1.6, -0.5, 2), vmin=1, vmax=3)

plt.scatter(data[:,0], data[:,1], c=labels.reshape(-1), cmap=plt.cm.brg, edgecolors='black')
plt.colorbar();
xx = np.linspace(-0.5,1.6,100)
yy1 = -weights1[0] / weights1[1] * xx - bias1 / weights1[1]
yy2 = -weights2[0] / weights2[1] * xx - bias2 / weights2[1]
yy3 = -weights3[0] / weights3[1] * xx - bias3 / weights3[1]
plt.plot(xx,yy1,xx,yy3,xx,yy2)
plt.ylim(-0.5,2)
plt.show()

Alternatively, we can view the training summaries in `TensorBoard`, `TensorFlow`'s display system.

We need to know where we've saved our training logs, in this case `logs/simple_class_...` where the `...` is the date and time.  We start `TensorBoard` by running

`tensorboard --logdir=logs/simple_class_...` 

in the terminal.

Normally after that you'd just go to the url it provides, but our JupyterHub setup is a little different, and instead we need to go to the link provided by this command

In [ ]:
from pylib.tensorboardcmd import tensorboard_cmd
logdir = './logs/simple_class_{}'.format(now_string)
tensorboard_cmd(logdir)

### Checkpointing

While our model is small and trained quickly, that is not the normal use case.  BigDL is meant for large networks that take hours to train.  As such, we'll want to have the model periodically save to disk, in case something goes wrong.  This is accomplished by adding a checkpoint as we did above:

```
fitter.set_checkpoint(optimizer.SeveralIteration(50), './logs/simple_class_checkpoint, 
                      isOverWrite=False))
```

It just takes a trigger and a directory to store the checkpoints.  An optional third argument, which defaults to True, switches between keeping just the latest checkpoint and keeping a separate file for each checkpoint.

We can load a checkpoint back in with a simple call. (You'll likely need to change the date/time string, BigDL creates it automatically.)

Enter the correct time and uncomment the following cells to run.

In [ ]:
#resumed_model = layer.Model.load('logs/simple_class_checkpoint/20180730_210437/model.50')

In [ ]:
#resumed_model.predict(np.array([[1,1]]))

You can also recover the state of the optimizer, as it's saved along with the checkpointed model.  You'll need to create a new optimizer object and pass this as the method to use it.  You'll need to change both the date and the name here, as it by default saves the optimizer with a unique identifier.

In [ ]:
#resumed_optim = optimizer.OptimMethod.load('logs/simple_class_checkpoint/20180730_210437/optimMethod-Sequential45607b9f.50')

## Classification of real-world data

A very commonly used data set for testing machine learners is MNIST.  It consists of a large number of small $28\times28$ pixel images of handwritten digits ($0$ to $9$), along with their labels.  It was the gold standard for a number of years, and is packaged with BigDL.  We'll be using it for some of our examples since it is convenient, well studied, and because we can do very well on it with modern techniques.

Since it's built in, we can grab it easily.  It will download the image set if it's not already on your computer.  

In [ ]:
from bigdl.dataset import mnist
from bigdl.util.common import Sample

mnist_path = "datasets/mnist"
(train_images, train_labels) = mnist.read_data_sets(mnist_path, "train")
(test_images, test_labels) = mnist.read_data_sets(mnist_path, "test")

# Note we'll need to add one to the label, since BigDL doesn't like zero
# as a label
mnist_train = sc.parallelize(zip(train_images, train_labels)).map(lambda x: Sample.from_ndarray(x[0],x[1]+1))

We can look at a few of these

In [ ]:
print("Shape: {}".format(train_images.shape))
print("Labels: {}".format(train_labels[:5]))
plt.imshow(train_images[:5].reshape([28*5,28]))

Note that shape: these are actually images, and are 2D (actually 3D, since there's that $\times1$ at the end).  Our requirements (so far) are that we have things that are vectors, i.e. 1D, so we'll need to fix that.  We can do it beforehand with `ndarray` manipulation in Spark or with the `Reshape` ability of BigDL.  We'll do the latter, since that's most likely what we'll do going forward.

In [ ]:
mnist_model = layer.Sequential()
mnist_model.add(layer.Reshape([28*28]))
mnist_model.forward(train_images[0]).shape

### Exercise: MNIST

Now that we've got a nice vector, build a logistic regressor on these images.  You won't be able to do amazingly well, but you'll be able to beat the pants off the "random guess" model (10% accuracy).  We'll do better as we learn more about neural networks.  Feel free to just add to the `mnist_model`, or create your own.  Also feel free to experiment with `CrossEntropyCriterion` without softmax.

## Things to watch out for

BigDL is still in development, so there are a few things to be careful about.  

Every worker gets a full copy of the model, its parameters, and its derivative that it must maintain in memory.  This depends on minibatch size as well.  If our model is too big, we usually get a Java Out of Memory error when we run the optimizer, but not always.  It can manifest as less clear errors, or as no error at all.  In the latter case, we'll just get a hanging run.  This can be a bit hard to diagnose, but we can see on `top` or `htop` that we have no significant CPU usage in that case.  Alternatively, you will see a failure in the Spark monitor UI at port 4040, but we don't have access to that port on these machines.

Large data sets don't always work correctly.  They can actually bring down the Spark kernel, which will result in very strange errors or complaints about connection failures.  This may be an issue more with the way it is interacting with Jupyter.

The documentation is still a work in progress.  However, as the interface is modeled after Torch, the Torch documentation is often quite helpful in determining what does what.  But be aware that function parameters are often named slightly differently.

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*